# Analyze Mon Valley property assessment appeals for 2022S and 2023

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
db = create_engine('postgresql://jon@localhost:5432/propertydb')

In [3]:
allappeals = pd.read_sql_query("""SELECT DISTINCT(parcelid)
FROM appeals
WHERE filer = 'PROPERTY OWNER'
AND appealperiod IN ('2022S', '2023');""", db)

### Select waterfront parcels, appeals by property owner

In [4]:
mjwaterfrontproperties = pd.read_sql_query("""SELECT DISTINCT(parcelid), municode, munidesc, fairmarkettotal::int, localtotal::int, parcelboundaries.wkt
FROM assessments
JOIN parcelboundaries
ON assessments.parcelid = parcelboundaries.pin
WHERE taxcode = 'T'
AND municode IN ('834', '840', '863')
AND (changenoticeaddress1 LIKE '20%%S%%CLARK%%ST%%' OR changenoticeaddress1 LIKE '180%%MICHIGAN%%AVE%%');""", db)

In [5]:
mjwaterfrontproperties['appeal'] = mjwaterfrontproperties.parcelid.isin(allappeals.parcelid)

mjwaterfrontproperties['fill'] = ''
mjwaterfrontproperties.loc[mjwaterfrontproperties.appeal, 'fill'] = '#fa8c00'

In [6]:
mjwaterfrontproperties[mjwaterfrontproperties.appeal].fairmarkettotal.sum() / mjwaterfrontproperties.fairmarkettotal.sum()

0.7474140039266717

In [7]:
mjwaterfrontproperties.to_csv('output/waterfront-local.csv', index=False)

### Select U.S. Steel parcels, appeals by property owner

In [8]:
steelproperties = pd.read_sql_query("""SELECT DISTINCT(assessments.parcelid), propertyhousenum, propertyaddress, municode, munidesc, fairmarkettotal::int, parcelcentroids.lat, parcelcentroids.lon
FROM assessments
JOIN parcelcentroids
ON assessments.parcelid = parcelcentroids.parcelid
WHERE taxcode = 'T'
AND (changenoticeaddress1 LIKE '1%%PPG%%PL%%2810%%' OR assessments.parcelid = '0657S00020000000');""", db)

In [9]:
steelproperties['appeal'] = steelproperties.parcelid.isin(allappeals.parcelid)

steelproperties['fill'] = ''
steelproperties.loc[steelproperties.appeal, 'fill'] = '#fa8c00'
steelproperties['symbol'] = 'locator'

steelproperties['title'] = ' '
steelproperties['tooltip'] = steelproperties.agg(lambda x: f"<b>Address:</b> {x['propertyhousenum']} {x['propertyaddress']}\n<b>Municipality:</b> {x['munidesc']}\n<b>Tax value:</b> {'${:,}'.format(x['fairmarkettotal'])}", axis=1)

In [10]:
steelproperties.to_csv('output/steel.csv', index=False)